In [ ]:
import os
import unicodedata

import pandas as pd
import pymssql
import missingno as msno

In [ ]:
!pip install missingno

In [ ]:
from collections import Counter
#from utils_ficha import clean_acents, null_verificator

In [ ]:
connection_b20 = pymssql.connect(host='10.10.33.92\SQL2008',
                             user='userweb',
                             port=1433,
                             password='$$W3BVPC2@17',
                             database='WEBVPC'
                            )

# FICHA COVID

In [ ]:
query_covid = """
    SELECT * FROM WEBVPC_COVID19 ORDER BY FECHA_CREACION DESC
"""
covid = pd.read_sql(
   query_covid , connection_b20
)
covid.head(10)

### ¿Cuantas fichas tengo?

In [ ]:
covid.shape

### ¿Que columnas se maneja?

In [ ]:
covid.columns

### ¿Cuales de estas columnas tienen nulos? ¿cuantos son estos? ¿En que porcentaje?

In [ ]:
_nulos = covid.isnull().sum()
_nulos[_nulos > 0]
pd.set_option('display.max_rows', 500)

null_verificator(covid)

In [ ]:
msno.matrix(covid,  labels=True)

## ANALISIS DETALLADO DE IMPACTO

## 1. Indicar si la pandemia COVID-19, ha tenido un impacto negativo en la gestión:

In [ ]:
null_verificator(pd.DataFrame(covid['PREGUNTA_1']))

In [ ]:
covid['PREGUNTA_1'].value_counts()   

#### NOTA: Pero de primera mano ya tenemos un indicador que nos dice como se ve el cliente frente al COVID o almenos que es lo que quiere transmitir de su situación

#### solo una cosa:   ¿QUE ES 1 Y QUE ES 2?

## 2_1 ¿Se encuentra actualmente operando?

In [ ]:
null_verificator(pd.DataFrame(covid['PREGUNTA_2_1']))

In [ ]:
covid['PREGUNTA_2_1'].value_counts()

#### NOTA: Que una empresa haya detenido operaciones por el COVID nos dá ya la intuición de que nos es factible otorgarle crédito y seria interesante cruzarla con una información de mora o clasificación.

## 2_2 ¿Se acogió a la suspensión perfecta de labores?

In [ ]:
null_verificator(pd.DataFrame(covid['PREGUNTA_2_2']))

#### NOTA: Permisos sin goze por tres meses a sus trabajadores con el fin de bajar al mínimo gatos por planilla indica que de por sí el rubro del negocio fue afectado por el COVID

## Revision de la pregunta 2 en general

#### SE INTUYE 1 = SI, 2 = NO

In [ ]:
all_2_question = covid[[col for col in covid.columns if col.startswith('PREGUNTA_2')]]
all_2_question[all_2_question['PREGUNTA_2_1'] == 1].head(3)

In [ ]:
all_2_question[all_2_question['PREGUNTA_2_1'] == 2].head(3)

#### NOTA: Las subpreguntas 2_x_x no depende de la primera, entonces  ¿Porqué agrauparlas en una sola pregunta?

## 2_3 ¿Redujo el personal?

In [ ]:
null_verificator(pd.DataFrame(covid['PREGUNTA_2_3_1']))

In [ ]:
all_2_question[all_2_question['PREGUNTA_2_3_2'].isnull()]['PREGUNTA_2_3_1'].value_counts()

#### ¿Quien es?

In [ ]:
covid[
    (covid['PREGUNTA_2_3_2'].isnull()) & (covid['PREGUNTA_2_3_1'] == 1)
]

NOTA: Detalle mostrado a revisar / El reducir personal ya es una acción drástica que da a notar que la empresa se prepara para una nueva realidad, se podría tener un treshold para el grado y ver como cruza con otras varibales

## 3. Otras medidas administrativas y financieras que se están toman o tomarán para afrontar la coyuntura y poder mitigar los efectos del COVID-19 adversos sobre el negocio:

In [ ]:
null_verificator(pd.DataFrame(covid['PREGUNTA_3']))

#### NOTA: solo 6.6% de nulos, con lo cual se asumiría que en esos casos no se han preocupado en tomar otras medidas para sobrepasar la coyuntura. De no ser así se debería alertar la necesidad de completar este campo

In [ ]:
covid['PREGUNTA_3'] = covid['PREGUNTA_3'].apply(clean_acents)

In [ ]:
covid['PREGUNTA_3']

In [ ]:
all_text =  ' '.join([_ for _ in list(covid['PREGUNTA_3']) if _]).strip().split()
all_text = [_ for _ in all_text if len(_) > 3 and _ not in ['para', 'hasta', 'esta', 'como', 'tomo']]

    
conteo = Counter(all_text)

In [ ]:
conteo.most_common()[:20]

#### Nota: se puede tener en cuenta estas palabras mas frecuentes para alimentar o reforzar otra pregunta y capacitar a los ejecutivos para que sepan como influencia el uso de las palabras que escriben en la toma de decisiones

## 4. En qué porcentaje se ha reducido su ingreso mensual en lo que va del 2020

In [ ]:
null_verificator(pd.DataFrame(covid['PREGUNTA_4']))

NOTA: al tener esta pregunta limpia de nulos, nos garantiza manejar con el porcentaje de disminución del ingreso que al menos el cliente quiere trasnmitir

## 5. En cuánto espera proyectar ventas al cierre del 2020

In [ ]:
null_verificator(pd.DataFrame(covid['PREGUNTA_5_1']))

In [ ]:
covid[covid['PREGUNTA_5_1'] < 0].shape, covid[covid['PREGUNTA_5_1'] > 0].shape, covid[covid['PREGUNTA_5_1'] == 0].shape

#### ¿Quienes son?

In [ ]:
covid[covid['PREGUNTA_5_1'] == 0][['COD_UNICO', 'FECHA_REGISTRO']].head(3)

#### NOTA: En los casos con valor se deduce que no habrá perdida, en caso contrario revisar el llenado

In [ ]:
null_verificator(pd.DataFrame(covid['PREGUNTA_5_2']))

In [ ]:
covid[covid['PREGUNTA_5_2'] < 0].shape, covid[covid['PREGUNTA_5_2'] > 0].shape, covid[covid['PREGUNTA_5_2'] == 0].shape

#### ¿Quienes son?

In [ ]:
covid[covid['PREGUNTA_5_2'] == 0][['COD_UNICO', 'FECHA_REGISTRO']].head(3)

#### NOTA: En los casos con valor se deduce que no habrá perdida, en caso contrario revisar el llenado

In [ ]:
covid[
    (covid['PREGUNTA_5_2'].notnull()) & (covid['PREGUNTA_5_1'].isnull())
].head(3)

## 6. ¿Qué porcentaje o monto de su costo de ventas y gastos operativos es fijo?

## 11. ¿Cuenta con algún crédito Reactiva en el Sistema Financiero (Excepto IBK)?

In [ ]:
null_verificator(pd.DataFrame(covid['PREGUNTA_5_2']))

In [ ]:
all_11 = covid[[col for col in covid.columns if col.startswith('PREGUNTA_11')] + ['COD_UNICO']]
all_11.shape
all_11.head(3)

In [ ]:
_null_11 = all_11[all_11['PREGUNTA_11'] == 1].isnull().sum(axis=1)
_null_11[_null_11 == all_11.shape[1]-2]

In [ ]:
all_11.iloc[704]